In [1]:
#import libraries
import ximea 
from ximea import xiapi
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import time
import cv2
import PIL
from PIL import Image as im
import napari

In [2]:
#Create camera object
cam1 = xiapi.Camera();
#Create image object
img1 = xiapi.Image()
#Open device using the serial number obtained from ximeaCOP
cam1.open_device_by_SN('CAMAU2146005');

In [3]:
#Start acquisition
cam1.start_acquisition()

In [4]:
#SETTINGS
#Allows access to sensor feature value currently 
#selected by XI_PRM_SENSOR_FEATURE_SELECTOR
cam1.set_sensor_feature_value(1)
#Define exposure 
cam1.set_exposure(int(5000))
#Define framerate 
cam1.set_framerate(100)
fps = int(cam1.get_framerate())
fps

100

In [208]:
#FOCUS AND ILLUMINATION CALIBRATION

try:
    print('Starting calibration...')
    #read current time
    t0 = time.time()
    while True:
        #get images
        cam1.get_image(img1)
        
        
        
        #create numpy array for each camera
        data1 = img1.get_image_data_numpy()

        #show video with a timer
        font = cv2.FONT_HERSHEY_PLAIN
        text = '{:5.2f}'.format((time.time()-t0))
        cv2.putText(data1, text, (1850,50), font, 4, (255, 255, 255), 2)
        video_cal = cv2.resize(data1, (968, 608))
        cv2.imshow('VIDEO CALIBRATION', video_cal)
        
        #waits for a key event for a "delay" of 1ms
        cv2.waitKey(1)
                
#halt acquisition
except KeyboardInterrupt:
        cv2.destroyAllWindows()
        cv2.waitKey(0)

Starting calibration...


In [221]:
#VIDEO ACQUISITION
video_array1 = []
seconds = 30
timing =time.time() + seconds
try:
    t0 = time.time()
    print('Starting video recording...')
    while time.time() < timing:
        #get images
        cam1.get_image(img1)
        
        #create numpy array for each camera
        data_raw1 = img1.get_image_data_numpy()

        
        video_rez = cv2.resize(data_raw1, (968, 608))
        #font = cv2.FONT_HERSHEY_PLAIN
        #text = '{:5.2f}'.format((time.time()-t0))
        #cv2.putText(video_rez, text, (700,50), font, 4, (255, 255, 255), 2)



        cv2.imshow('RECORDING...', video_rez)
        video_array1.append(video_rez)

        #waits for a key event for a "delay" of 1ms
        cv2.waitKey(1)
        
#halt acquisition
except KeyboardInterrupt:
    cv2.destroyAllWindows()

Starting video recording...


In [7]:
#CROP ONLY ONCE!!!!!
#Get image from CAMERA1 as a numpy array
cam1.get_image(img1)
data_raw1 = img1.get_image_data_numpy()
#Select the points to crop the region of interest for acquisition. ONLY ONCE!!!!!
#Start with the following order: 
#(1) TOP-LEFT CORNER
#(2) DOWN-LEFT CORNER
#(3) DOWN-RIGHT CORNER
#(4) TOP-RIGHT CORNER
data_rez = cv2.resize(data_raw1, (968, 608))
viewer1=napari.Viewer()
viewer1.add_image(data_rez)

C:\Users\AdminUser\anaconda3\lib\site-packages\napari\_qt\__init__.py:53: UserWarning: 

napari was tested with QT library `>=5.12.3`.
The version installed is 5.9.7. Please report any issues with
this specific QT version at https://github.com/Napari/napari/issues.
  warn(message=warn_message)


<Image layer 'data_rez' at 0x222d5d066a0>

In [8]:
points1 = viewer1.layers['Points'].data
print(points1)

[[-2.89701219e-01  1.99704221e+02]
 [ 6.00351882e+02  1.99669563e+02]
 [ 6.00299117e+02  8.00043393e+02]
 [ 7.60777824e-02  7.99966599e+02]]


In [12]:
#points1[0,1] points1[3,0], points1[2,1] points1[1,0]

600.3518820053202

In [88]:
#CROP VERIFICATION ONLY ONCE!!!!!
data1 = im.fromarray(data_rez)
box1 = (200, 0, 800, 600)
cropped_image1 = data1.crop(box1)

#Convert images to grayscale
figs, plots = plt.subplots(1,2)
plots[0].imshow(data_raw1, cmap='gray', vmin=0, vmax=255)
plots[1].imshow(cropped_image1, cmap='gray', vmin=0, vmax=255)

NameError: name 'data_rez' is not defined

In [222]:
#CROP VIDEO
cropped_vid1 = []
box1 = (200, 0, 800, 600)
for i in range(len(video_array1)): 
    data_i1 = im.fromarray(video_array1[i])
    data1 = np.array(data_i1.crop(box1))
    cropped_vid1.append(data1)

#VIDEO PARAMETERS
frame_width_top = int(data1.shape[1])
frame_height_top = int(data1.shape[0])

frames = len(cropped_vid1)
print('total frames = ',frames)

video1 = np.array(cropped_vid1)
# FourCC is a 4-byte code used to specify the video codec.
fourcc = cv2.VideoWriter_fourcc(*'MP42')

total frames =  3480


In [223]:
#Syntax: cv2.VideoWriter( filename, fourcc, fps, frameSize )
output_video1 = cv2.VideoWriter('sf3s5_v10.mp4', fourcc, (len(video_array1)/seconds), (frame_width_top, frame_height_top))

for i in range(len(video1)):
    output_video1.write(video1[i])
 
output_video1.release()

In [36]:
#stop communication from both cameras
cam1.stop_acquisition()
cam1.close_device()